In [1]:
import numpy as np
load("sample_gauss.sage")
from enc_dec import np_encrypt, np_decrypt
import time
from g_inv import np_gen_g
from basis import np_mul_vec, np_mul_vec_ele, np_gen_poly
import pickle
from random import randint
import sys
import sage.rings as rings
sys.modules['sage.rings'] = rings

np.set_printoptions(threshold=np.inf)
def hash_func(n, k, q, z, h_):
    g = np_gen_g(n, k)
    id_g = np_mul_vec_ele(g, z, q)
    return ((id_g + h_) + q//2) % q - q//2


def scheme1(n, k, q, h_, z, a, B, mu, inv_B, u, message, fp):
    
    message = bin(message)
    arr = np.array(list(message[:1:-1]), dtype='int64')
    message_ = np.pad(arr, (0, n - len(arr)), mode='constant')
    
    hash_time = 0.0
    kg_time = 0.0
    enc_time = 0.0
    dec_time = 0.0
    time0 = time.time()
    h_id = hash_func(n, k, q, z, h_)
    time1 = time.time()
    h = a[1:]
#     print(f"id={z}", file=fp)
#     print(f"h_={h_}", file=fp)
#     print(f"h_id={h_id}", file=fp)
#     print(f"mpk size = {(sys.getsizeof(h)+sys.getsizeof(h_)+sys.getsizeof(u))*8/1024}",file=fp)
    f_id = np.vstack((h, h_id))
#     print(f"f_id={f_id}", file=fp)
    while True:
        time2 = time.time()
        e_id = np_sam_l(n, k, q, B, mu, inv_B, u, h_id)
#         print(f"e_id={e_id}", file=fp)
        time3 = time.time()
#         print(f"eid norm = {np.linalg.norm(e_id)}", file=fp)
#         print(f"sk size = {sys.getsizeof(e_id)*8/1024}",file=fp)
        time4 = time.time()
        c0, c1, e1 = np_encrypt(n, k, q, u, f_id, message_, fp)
#         print(f"c0={c0}", file=fp)
#         print(f"c1={c1}", file=fp)
        time5 = time.time()
#         print(f"ct size = {(sys.getsizeof(c0)+sys.getsizeof(c1))*8/1024}",file=fp)
#         print(f"e1 * eid = {np.linalg.norm(np_mul_vec(e1, e_id, q))}", file=fp)
        time6 = time.time()
        mes = np_decrypt(n, q, e_id, c0, c1)
        time7 = time.time()
#         print(f"mes={mes}", file=fp)
#         print(f"mes={message_}", file=fp)
        
        if (message_ == mes).all():
            print(f"correct decryption!", file=fp)
        
        hash_time = time1 - time0
        kg_time = time3 - time2
        enc_time = time5 - time4
        dec_time = time7 - time6
#         print(f"hash time = {hash_time}", file=fp)
        print(f"keygen time = {kg_time}", file=fp)
        print(f"encrypt time = {enc_time}", file=fp)
        print(f"decrypt time = {dec_time}", file=fp)
        fp.flush()

        return mes
    

def main():
    n = 2**8
    q = 8331299
    k = 8

    a = pickle.load(open(f"data/param_2/a.pkl", "rb"))
    B = pickle.load(open(f"data/param_2/B.pkl", "rb"))
#     Ta_gs = pickle.load(open(f"data/scheme1/data{n}_/Ta_gs.pkl", "rb"))
    mu = pickle.load(open(f"data/param_2/mu.pkl", "rb"))
    inv_B = pickle.load(open(f"data/param_2/inv_B.pkl", "rb"))
    print("import data success ! ")

    z = [randint(0, 1) for i in range(n)]
    h_ = np.zeros((k, n), dtype=object)
    for i in range(k):
        h_[i] = np_gen_poly(n, q)
    # writing to txt
    with open(f'result/param_2', 'w') as fp:
        u = np_gen_poly(n, q)
        for message in [randint(0, 2 ** n - 1) for i in range(1)]:
            scheme1(n, k, q, h_, z, a, B, mu, inv_B, u, message, fp)

if __name__ == '__main__':
    main()


import data success ! 
